In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [4]:
MY_PATH = "drive/MyDrive/COMS E6998 PDL Sys Perf/Project/"
os.chdir(MY_PATH)

In [5]:
os.getcwd()

'/content/drive/MyDrive/COMS E6998 PDL Sys Perf/Project'

# Display image

In [14]:
test_image = "dl-project/test/destroyed/santa-rosa-wildfire_00000204_post_disaster.png"

In [15]:
from PIL import Image

image = Image.open(test_image)

In [16]:
image

Output hidden; open in https://colab.research.google.com to view.

# Amazon Rekognition Model

In [ ]:
!pip install --upgrade boto3

## Start model

In [13]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition', region_name='us-east-1', 
                        aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def main():
    project_arn='arn:aws:rekognition:us-east-1:639440887017:project/disaster-classify-project/1650477978851'
    model_arn='arn:aws:rekognition:us-east-1:639440887017:project/disaster-classify-project/version/disaster-classify-project.2022-05-05T10.10.24/1651759824975'
    min_inference_units=1 
    version_name='disaster-classify-project.2022-05-05T10.10.24'
    start_model(project_arn, model_arn, version_name, min_inference_units)

if __name__ == "__main__":
    main()

Starting model: arn:aws:rekognition:us-east-1:639440887017:project/disaster-classify-project/version/disaster-classify-project.2022-05-05T10.10.24/1651759824975
Status: RUNNING
Message: The model is running.
Done...


## Make inference / Analyze image

In [18]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

def show_custom_labels(model,bucket,photo, min_confidence):
    client=boto3.client('rekognition', region_name='us-east-1', 
                        aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

    #Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)

    print(response['CustomLabels'])
    return response, len(response['CustomLabels'])

def main():

    bucket='custom-labels-console-us-east-1-210b052f46'
    # photo='test/destroyed/hurricane-harvey_00000298_post_disaster.png'
    photo=test_image[test_image.index('/')+1:]  # test/destroyed/santa-rosa-wildfire_00000204_post_disaster.png
    model='arn:aws:rekognition:us-east-1:639440887017:project/disaster-classify-project/version/disaster-classify-project.2022-05-05T10.10.24/1651759824975'
    min_confidence=95

    response, label_count = show_custom_labels(model,bucket,photo, min_confidence)
    print("Custom labels detected: " + str(label_count))


if __name__ == "__main__":
    main()

[{'Name': 'destroyed', 'Confidence': 99.98900604248047}]
Custom labels detected: 1


## Stop training

In [19]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import time


def stop_model(model_arn):

    client=boto3.client('rekognition', region_name='us-east-1', 
                        aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

    print('Stopping model:' + model_arn)

    #Stop the model
    try:
        response=client.stop_project_version(ProjectVersionArn=model_arn)
        status=response['Status']
        print ('Status: ' + status)
    except Exception as e:  
        print(e)  

    print('Done...')
    
def main():
    
    model_arn='arn:aws:rekognition:us-east-1:639440887017:project/disaster-classify-project/version/disaster-classify-project.2022-05-05T10.10.24/1651759824975'
    stop_model(model_arn)

if __name__ == "__main__":
    main() 

Stopping model:arn:aws:rekognition:us-east-1:639440887017:project/disaster-classify-project/version/disaster-classify-project.2022-05-05T10.10.24/1651759824975
Status: STOPPING
Done...


# Appendix

## Initialize AWS access keys (these keys expired after the demo)

In [12]:
ACCESS_KEY_ID = "XXXXXXXXXXXXXXXXXXX"
SECRET_ACCESS_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"